# Hubspot Integration

In [96]:
import urllib3, json, requests
from bs4 import BeautifulSoup
import urllib.request as urllib2
import pandas as pd

In [28]:
api_key =  "?hapikey=" + hubspot_api_key
hubspot_api_url = "http://api.hubapi.com"
http = urllib3.PoolManager()


In [94]:
def getTotalNumberOfContacts():
    # First, we build the correct url

    xulr = "/contacts/v1/contacts/statistics"

    url = hubspot_api_url + xulr + api_key
    # Now we use urllib3 to open the url and read it
    #print(url)
    #response = requests.get(url)
    #page = urllib2.urlopen(url)
    #soup = BeautifulSoup(response.content, "html.parser")
    response = urllib2.urlopen(url).read()

    #soup = BeautifulSoup(page)
    statistics = json.loads(response)
    
    print(statistics["contacts"])
    


In [95]:
getTotalNumberOfContacts()

37187


In [118]:
def get_recently_created_contacts():
    
    xulr = "/contacts/v1/lists/all/contacts/recent"

    url = hubspot_api_url + xulr + api_key +"&timeOffset=2"
    # Now we use urllib3 to open the url and read it
    #print(url)
    response = urllib2.urlopen(url).read()
    #response = requests.get(url)
    #soup = BeautifulSoup(response.content, "html.parser")
    statistics = json.loads(response)
    df = pd.DataFrame(statistics["contacts"])
    lista = statistics["contacts"]    
    print(l0)

In [119]:
get_recently_created_contacts()

<class 'dict'>
{'addedAt': 1512394575789, 'vid': 45816011, 'canonical-vid': 45816011, 'merged-vids': [], 'portal-id': 556697, 'is-contact': True, 'profile-token': 'AO_T-mMu-9DJk8sRfdl7gV1r-tz2Ahdfso6aQN87CHtOBeDi9rwt8dT5SVT8CHFRe98XZEEBaRZ9TtpmRbZvyXP7u1y9DoXY3CMxbbg_t4ZsdFtllVXZuaLUjsRceTXIWIIyR1ogvEeO', 'profile-url': 'https://app.hubspot.com/contacts/556697/lists/public/contact/_AO_T-mMu-9DJk8sRfdl7gV1r-tz2Ahdfso6aQN87CHtOBeDi9rwt8dT5SVT8CHFRe98XZEEBaRZ9TtpmRbZvyXP7u1y9DoXY3CMxbbg_t4ZsdFtllVXZuaLUjsRceTXIWIIyR1ogvEeO/', 'properties': {'firstname': {'value': 'Dawid'}, 'lastmodifieddate': {'value': '1512396394016'}, 'company': {'value': 'Lancer'}, 'lastname': {'value': 'Sokolski'}}, 'form-submissions': [{'conversion-id': 'f6039e2e-620c-4311-8692-e0d859ec13b3', 'timestamp': 1512394575789, 'form-id': '39a44f3b-bdb2-406e-a78c-c304a4a53c07', 'portal-id': 556697, 'title': 'Panel Kontakt.io - create an account - TEST', 'form-type': 'HUBSPOT', 'meta-data': []}], 'identity-profiles': [{'vid':